In [76]:
import pandas as pd
import json
import numpy as np
import datetime

In [78]:
for i in range (5):
    with open(f'../data/data{i}.json') as f:
            d = json.load(f)
        
    df = pd.json_normalize(data=d['drugSuccessPredictions'], record_path='drugPrograms', meta=['drugId', 'displayName'])

    filtered_data = [
        {
            "drugId": dp["drugId"],
            "displayName": dp["displayName"],
            "drugPrograms": [prog 
                            for prog in dp["drugPrograms"] 
                            if (
                                    any(
                                            (phase["phase"] == "Pre-Reg" and 
                                            pd.Timestamp(phase["endDate"]).date() < datetime.date(2024, 5, 1) and 
                                            pd.Timestamp(phase["endDate"]).date() > datetime.date(2023, 11, 1)) 
                                            for phase in prog["phaseTimelines"]
                                        ))]
        }
        for dp in d["drugSuccessPredictions"]
    ]

    df = pd.json_normalize(
            data=filtered_data, 
            record_path=['drugPrograms', 'phaseTimelines'], 
            meta=[
                    'drugId', 
                    'displayName', 
                    ['drugPrograms', 'indication', 'name'], 
                    ['drugPrograms', 'company', 'name'], 
                    ['drugPrograms', 'country', 'name'], 
                    ['drugPrograms', 'phaseHighestStatus'], 
                    ['drugPrograms', 'probabilityOfSuccess']])

    selected_columns = ['drugId', 'displayName', 'drugPrograms.indication.name', 'drugPrograms.company.name', 'drugPrograms.country.name', 'drugPrograms.probabilityOfSuccess', 'phase', 'startDate', 'endDate', 'successPercentage']
    df = df[selected_columns]
    df = df.rename(columns={'drugPrograms.indication.name': 'indication', 'drugPrograms.company.name': 'company', 'drugPrograms.country.name': 'country', 'drugPrograms.probabilityOfSuccess': 'probabilityOfSuccess'})
    df = df[df['country'] == 'US']
    df = df[df['probabilityOfSuccess'].str.rstrip('%').astype('float') / 100 > 0.9]
    df_less = df[df['phase'] == 'Pre-Reg']
    df_less.to_csv(f'../results/less/data{i}.csv')
    df.to_csv(f'../results/more/data{i}.csv')


/var/folders/xr/k8w2v67j76n61jc0bhbwyjz00000gn/T/ipykernel_37307/1750479401.py:16: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.
  pd.Timestamp(phase["endDate"]).date() < datetime.date(2024, 5, 1) and


In [75]:
dfs_less = []
for i in range (5):
    df = pd.read_csv(f'../results/less/data{i}.csv')
    dfs_less.append(df)
combined_df = pd.concat(dfs_less, ignore_index=True).drop("Unnamed: 0", axis=1) 
combined_df.to_csv('../results/final/selected_drug.csv')

dfs_more = []
for i in range (5):
    df = pd.read_csv(f'../results/more/data{i}.csv')
    dfs_more.append(df)
combined_df = pd.concat(dfs_more, ignore_index=True).drop("Unnamed: 0", axis=1) 
combined_df.to_csv('../results/final/selected_drug_phases.csv')